In [ ]:
# Step 1 — Upload your Kaggle API key (kaggle.json)
from google.colab import files
files.upload()  # 👉 Upload kaggle.json from your local system
# Step 2 — Move kaggle.json to the correct folder and set permissions
!mkdir -p ~/.kaggle 
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# Step 3 — Download the Coral Reefs Images dataset from Kaggle
!kaggle datasets download -d asfarhossainsitab/coral-reefs-images
# Step 4 — Unzip the dataset into a working directory
!unzip -q coral-reefs-images.zip -d ./coral_reef_dataset


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/asfarhossainsitab/coral-reefs-images
License(s): apache-2.0
 95% 674M/711M [00:03<00:00, 139MB/s] 
100% 711M/711M [00:03<00:00, 213MB/s]


In [ ]:
!pip install tensorflow matplotlib scikit-learn opencv-python pillow


In [ ]:
# coral_classifier_with_gradcam.py
# Python 3.8+, TensorFlow 2.x
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import cv2
from sklearn.metrics import classification_report, confusion_matrix
import itertools
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory





In [ ]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import json
import os
import cv2

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define constants
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.0001

# Define paths
train_dir = '/content/coral_reef_dataset/Coral Reef Images/train'
validation_dir = '/content/coral_reef_dataset/Coral Reef Images/valid'
model_save_path = 'coral_reef_resnet50_model.h5'
model_pkl_path = 'coral_reef_resnet50_model.pkl'
history_pkl_path = 'training_history.pkl'

# Create directory for saving outputs
os.makedirs('outputs', exist_ok=True)
os.makedirs('gradcam_visualizations', exist_ok=True)

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Load data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# Get class names
class_names = list(train_generator.class_indices.keys())
print(f"Classes: {class_names}")
print(f"Class indices: {train_generator.class_indices}")

# Save class indices
with open('class_indices.pkl', 'wb') as f:
    pickle.dump(train_generator.class_indices, f)

# Create ResNet50 model
def create_resnet50_model(num_classes=1):
    """
    Create a ResNet50 model for binary classification
    """
    # Load pre-trained ResNet50
    base_model = ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
    )

    # Freeze base model layers initially
    base_model.trainable = False

    # Build the model
    inputs = layers.Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)

    return model, base_model

# Create the model
print("Creating ResNet50 model...")
model, base_model = create_resnet50_model()

# Compile the model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

# Print model summary
print("\nModel Summary:")
model.summary()

# Define callbacks
callbacks = [
    ModelCheckpoint(
        model_save_path,
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

# Train the model (Phase 1: Frozen base model)
print("\n" + "="*50)
print("Phase 1: Training with frozen base model")
print("="*50 + "\n")

history_phase1 = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    callbacks=callbacks,
    verbose=1
)

# Fine-tuning (Phase 2: Unfreeze some layers)
print("\n" + "="*50)
print("Phase 2: Fine-tuning - Unfreezing top layers")
print("="*50 + "\n")

# Unfreeze the base model
base_model.trainable = True

# Freeze all layers except the last 50
for layer in base_model.layers[:-50]:
    layer.trainable = False

# Recompile with a lower learning rate
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE/10),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()]
)

# Continue training
history_phase2 = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks,
    initial_epoch=history_phase1.epoch[-1],
    verbose=1
)

# Combine training histories
combined_history = {
    'accuracy': history_phase1.history['accuracy'] + history_phase2.history['accuracy'],
    'val_accuracy': history_phase1.history['val_accuracy'] + history_phase2.history['val_accuracy'],
    'loss': history_phase1.history['loss'] + history_phase2.history['loss'],
    'val_loss': history_phase1.history['val_loss'] + history_phase2.history['val_loss'],
    'precision': history_phase1.history['precision'] + history_phase2.history['precision_1'],
    'val_precision': history_phase1.history['val_precision'] + history_phase2.history['val_precision_1'],
    'recall': history_phase1.history['recall'] + history_phase2.history['recall_1'],
    'val_recall': history_phase1.history['val_recall'] + history_phase2.history['val_recall_1']
}

# Save training history
with open(history_pkl_path, 'wb') as f:
    pickle.dump(combined_history, f)

print(f"\nTraining history saved to {history_pkl_path}")

# Load best model
model = keras.models.load_model(model_save_path)

# Evaluate the model
print("\n" + "="*50)
print("Final Model Evaluation")
print("="*50 + "\n")

loss, accuracy, precision, recall = model.evaluate(validation_generator)
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")
print(f"Validation Precision: {precision:.4f}")
print(f"Validation Recall: {recall:.4f}")
print(f"Validation F1-Score: {f1_score:.4f}")

# Generate predictions
validation_generator.reset()
y_true = validation_generator.classes
y_pred_probs = model.predict(validation_generator, verbose=1)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

# Classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:")
print(cm)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('outputs/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.close()
print("Confusion matrix saved to outputs/confusion_matrix.png")

# Plot training history
def plot_training_history(history, save_path='outputs/training_history.png'):
    """Plot training and validation metrics"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # Accuracy
    axes[0, 0].plot(history['accuracy'], label='Train Accuracy')
    axes[0, 0].plot(history['val_accuracy'], label='Val Accuracy')
    axes[0, 0].set_title('Model Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True)

    # Loss
    axes[0, 1].plot(history['loss'], label='Train Loss')
    axes[0, 1].plot(history['val_loss'], label='Val Loss')
    axes[0, 1].set_title('Model Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True)

    # Precision
    axes[1, 0].plot(history['precision'], label='Train Precision')
    axes[1, 0].plot(history['val_precision'], label='Val Precision')
    axes[1, 0].set_title('Model Precision')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Precision')
    axes[1, 0].legend()
    axes[1, 0].grid(True)

    # Recall
    axes[1, 1].plot(history['recall'], label='Train Recall')
    axes[1, 1].plot(history['val_recall'], label='Val Recall')
    axes[1, 1].set_title('Model Recall')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Recall')
    axes[1, 1].legend()
    axes[1, 1].grid(True)

    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Training history plots saved to {save_path}")

plot_training_history(combined_history)

# Save model to pickle file
print(f"\nSaving model to {model_pkl_path}...")
with open(model_pkl_path, 'wb') as f:
    pickle.dump({
        'model_architecture': model.to_json(),
        'model_weights': model.get_weights(),
        'class_indices': train_generator.class_indices,
        'input_shape': (IMG_WIDTH, IMG_HEIGHT, 3)
    }, f)
print(f"Model saved to {model_pkl_path}")

# Save model architecture to JSON
model_json = model.to_json()
with open('coral_reef_model_architecture.json', 'w') as json_file:
    json_file.write(model_json)
print("Model architecture saved to coral_reef_model_architecture.json")

# =====================================================
# GradCAM Implementation
# =====================================================

def get_img_array(img_path, size=(IMG_WIDTH, IMG_HEIGHT)):
    """Load and preprocess image"""
    img = load_img(img_path, target_size=size)
    array = img_to_array(img)
    array = np.expand_dims(array, axis=0)
    array = array / 255.0
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    """
    Generate GradCAM heatmap
    """
    # Create a model that maps the input image to the activations of the last conv layer
    grad_model = keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # Gradient of the output neuron with regard to the output feature map
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # Vector of mean intensity of the gradient over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Multiply each channel by "how important this channel is"
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # Normalize the heatmap
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def save_and_display_gradcam(img_path, heatmap, output_path, alpha=0.4):
    """
    Overlay GradCAM heatmap on original image and save
    """
    # Load the original image
    img = load_img(img_path)
    img = img_to_array(img)

    # Rescale heatmap to range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = plt.cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.utils.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.utils.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.utils.array_to_img(superimposed_img)

    # Create a figure with subplots
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # Original image
    axes[0].imshow(keras.utils.array_to_img(img))
    axes[0].set_title('Original Image', fontsize=14, fontweight='bold')
    axes[0].axis('off')

    # Heatmap
    axes[1].imshow(jet_heatmap / 255.0)
    axes[1].set_title('GradCAM Heatmap', fontsize=14, fontweight='bold')
    axes[1].axis('off')

    # Superimposed
    axes[2].imshow(superimposed_img)
    axes[2].set_title('GradCAM Overlay', fontsize=14, fontweight='bold')
    axes[2].axis('off')

    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

    return superimposed_img

def generate_gradcam_for_image(img_path, model, class_indices, output_dir='gradcam_visualizations'):
    """
    Generate GradCAM visualization for a single image
    """
    # Get the name of the last convolutional layer in ResNet50
    last_conv_layer_name = 'conv5_block3_out'  # Last conv layer in ResNet50

    # Prepare image
    img_array = get_img_array(img_path)

    # Make prediction
    preds = model.predict(img_array, verbose=0)
    pred_class = int(preds[0] > 0.5)
    confidence = preds[0][0] if pred_class == 1 else 1 - preds[0][0]

    # Get class names
    class_names_dict = {v: k for k, v in class_indices.items()}
    predicted_class_name = class_names_dict[pred_class]

    # Generate heatmap
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

    # Create output filename
    img_filename = os.path.basename(img_path)
    output_path = os.path.join(output_dir, f'gradcam_{img_filename}')

    # Save visualization
    save_and_display_gradcam(img_path, heatmap, output_path)

    print(f"GradCAM for {img_filename}:")
    print(f"  Predicted: {predicted_class_name} (Confidence: {confidence:.2%})")
    print(f"  Saved to: {output_path}")

    return predicted_class_name, confidence

# Generate GradCAM for sample images from validation set
print("\n" + "="*50)
print("Generating GradCAM Visualizations")
print("="*50 + "\n")

# Get sample images from each class
sample_images = []
for class_name in class_names:
    class_dir = os.path.join(validation_dir, class_name)
    if os.path.exists(class_dir):
        images = [os.path.join(class_dir, img) for img in os.listdir(class_dir)
                 if img.lower().endswith(('.png', '.jpg', '.jpeg'))][:5]  # Get 5 samples per class
        sample_images.extend(images)

# Generate GradCAM for sample images
for img_path in sample_images:
    try:
        generate_gradcam_for_image(img_path, model, train_generator.class_indices)
    except Exception as e:
        print(f"Error processing {img_path}: {e}")

print(f"\nGradCAM visualizations saved to 'gradcam_visualizations/' directory")

# =====================================================
# Utility function to load saved model from pickle
# =====================================================

def load_model_from_pickle(pkl_path):
    """
    Load model from pickle file
    """
    with open(pkl_path, 'rb') as f:
        model_data = pickle.load(f)

    # Reconstruct model
    model = keras.models.model_from_json(model_data['model_architecture'])
    model.set_weights(model_data['model_weights'])

    return model, model_data['class_indices']

# Save final metrics
final_metrics = {
    'accuracy': float(accuracy),
    'precision': float(precision),
    'recall': float(recall),
    'f1_score': float(f1_score),
    'loss': float(loss),
    'confusion_matrix': cm.tolist(),
    'class_names': class_names
}

with open('final_metrics.json', 'w') as f:
    json.dump(final_metrics, f, indent=4)

print("\n" + "="*50)
print("Training Complete!")
print("="*50)
print(f"\nSaved files:")
print(f"  - Model (.h5): {model_save_path}")
print(f"  - Model (.pkl): {model_pkl_path}")
print(f"  - Training history: {history_pkl_path}")
print(f"  - Class indices: class_indices.pkl")
print(f"  - Model architecture: coral_reef_model_architecture.json")
print(f"  - Final metrics: final_metrics.json")
print(f"  - Confusion matrix: outputs/confusion_matrix.png")
print(f"  - Training plots: outputs/training_history.png")
print(f"  - GradCAM visualizations: gradcam_visualizations/")

print("\n" + "="*50)
print("Model Performance Summary")
print("="*50)
print(f"Accuracy:  {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall:    {recall:.2%}")
print(f"F1-Score:  {f1_score:.2%}")
print("="*50)

Found 9662 images belonging to 2 classes.
Found 463 images belonging to 2 classes.
Classes: ['Bleached', 'Healthy']
Class indices: {'Bleached': 0, 'Healthy': 1}
Creating ResNet50 model...
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

Model Summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,153,473 (92.14 MB)

 Trainable params: 561,665 (2.14 MB)

...


Epoch 39: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.
302/302 ━━━━━━━━━━━━━━━━━━━━ 155s 513ms/step - accuracy: 0.8899 - loss: 0.2763 - precision_1: 0.9018 - recall_1: 0.8700 - val_accuracy: 0.8963 - val_loss: 0.3403 - val_precision_1: 0.8788 - val_recall_1: 0.9103 - learning_rate: 5.0000e-06
Epoch 40/50
302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 508ms/step - accuracy: 0.8891 - loss: 0.2655 - precision_1: 0.8954 - recall_1: 0.8714
Epoch 40: val_accuracy improved from 0.89633 to 0.90065, saving model to coral_reef_resnet50_model.h5


302/302 ━━━━━━━━━━━━━━━━━━━━ 162s 538ms/step - accuracy: 0.8891 - loss: 0.2655 - precision_1: 0.8954 - recall_1: 0.8714 - val_accuracy: 0.9006 - val_loss: 0.3690 - val_precision_1: 0.8672 - val_recall_1: 0.9372 - learning_rate: 2.5000e-06
Epoch 41/50
302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 508ms/step - accuracy: 0.8934 - loss: 0.2768 - precision_1: 0.8993 - recall_1: 0.8754
Epoch 41: val_accuracy did not improve from 0.90065
302/302 ━━━━━━━━━━━━━━━━━━━━ 155s 514ms/step - accuracy: 0.8934 - loss: 0.2768 - precision_1: 0.8993 - recall_1: 0.8753 - val_accuracy: 0.8855 - val_loss: 0.3208 - val_precision_1: 0.9126 - val_recall_1: 0.8430 - learning_rate: 2.5000e-06
Epoch 42/50
302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 500ms/step - accuracy: 0.8843 - loss: 0.2824 - precision_1: 0.8925 - recall_1: 0.8675
Epoch 42: val_accuracy did not improve from 0.90065
302/302 ━━━━━━━━━━━━━━━━━━━━ 153s 506ms/step - accuracy: 0.8843 - loss: 0.2824 - precision_1: 0.8925 - recall_1: 0.8675 - val_accuracy: 0.8704 - val_loss: 0.3


Final Model Evaluation

15/15 ━━━━━━━━━━━━━━━━━━━━ 10s 256ms/step - accuracy: 0.8816 - loss: 0.5526 - precision_1: 0.4147 - recall_1: 0.5332
Validation Loss: 0.3690
Validation Accuracy: 0.9006
Validation Precision: 0.8672
Validation Recall: 0.9372
Validation F1-Score: 0.9009
15/15 ━━━━━━━━━━━━━━━━━━━━ 10s 425ms/step

Classification Report:
              precision    recall  f1-score   support

    Bleached       0.94      0.87      0.90       240
     Healthy       0.87      0.94      0.90       223

    accuracy                           0.90       463
   macro avg       0.90      0.90      0.90       463
weighted avg       0.90      0.90      0.90       463


Confusion Matrix:
[[208  32]
 [ 14 209]]
Confusion matrix saved to outputs/confusion_matrix.png
Training history plots saved to outputs/training_history.png

Saving model to coral_reef_resnet50_model.pkl...
Model saved to coral_reef_resnet50_model.pkl
Model architecture saved to coral_reef_model_architecture.json

Generating Gr

/tmp/ipython-input-445316010.py:425: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_class = int(preds[0] > 0.5)


Error processing /content/coral_reef_dataset/Coral Reef Images/valid/Bleached/youtube-816_jpg.rf.3d54489db652e1db5d12d8c3e689aac6.jpg: No such layer: conv5_block3_out. Existing layers are: ['input_layer_1', 'resnet50', 'global_average_pooling2d', 'batch_normalization', 'dense', 'dropout', 'dense_1', 'dropout_1', 'dense_2'].
Error processing /content/coral_reef_dataset/Coral Reef Images/valid/Bleached/youtube-901_jpg.rf.e35d9db41c50c029942da5f744ef2772.jpg: No such layer: conv5_block3_out. Existing layers are: ['input_layer_1', 'resnet50', 'global_average_pooling2d', 'batch_normalization', 'dense', 'dropout', 'dense_1', 'dropout_1', 'dense_2'].
Error processing /content/coral_reef_dataset/Coral Reef Images/valid/Bleached/youtube-1024_jpg.rf.43ea89de789c53dff14ccc1b810e593a.jpg: No such layer: conv5_block3_out. Existing layers are: ['input_layer_1', 'resnet50', 'global_average_pooling2d', 'batch_normalization', 'dense', 'dropout', 'dense_1', 'dropout_1', 'dense_2'].
Error processing /con

In [ ]:
from google.colab import files
files.download("coral_reef_resnet50_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("coral_reef_resnet50_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>